Native Python client for AliEn: alien.py
==============================

The standard command line utilities for AliEn are implemented in python anyways, so for interacting with the grid (e.g. file catalog, job submission), this is probably the easier solution.

Here we do a basic check.

In [2]:
import alienpy.alien as alien

alien.setup_logging()
try:
    j = alien.AliEn()
    j.ProcessMsg('whoami -v')

except SystemExit:
    print("ERROR initializing AliEn session")
    print("Make sure you have a valid token by opening a SWAN terminal and running `alien-token-init`")
    


Username: tdietel
Role: tdietel (other roles: trddrop)
Full name: Thomas Dietel
Email: Tom.Dietel@cern.ch
Certificate DN: OU=tdietel, CN=tdietel, CN=Users, O=AliEn2, C=ch (expiring on 2022-05-21T16:50:56+02:00[Europe/Paris], which is in 30d 23:59)
Connected from: 188.184.38.69
Home directory: /alice/cern.ch/user/t/tdietel/


Querying the file catalog
-----------------------------

To look around, one can run simple AliEn commands and see the output (like on a terminal). 

In [3]:
j.ProcessMsg('ls /alice/data/2021/OCT_TRD/504428/raw')

o2_ctf_run00504428_orbit0000000124_tf0000000001.root
o2_ctf_run00504428_orbit0000001020_tf0000000008.root
o2_ctf_run00504428_orbit0000002428_tf0000000019.root
o2_ctf_run00504428_orbit0000002812_tf0000000022.root
o2_ctf_run00504428_orbit0000005756_tf0000000045.root
o2_ctf_run00504428_orbit0000006268_tf0000000049.root
o2_ctf_run00504428_orbit0000007036_tf0000000055.root
o2_ctf_run00504428_orbit0000127356_tf0000000995.root
o2_ctf_run00504428_orbit0000127612_tf0000000997.root
o2_ctf_run00504428_orbit0000127868_tf0000000999.root
o2_ctf_run00504428_orbit0000128508_tf0000001004.root
o2_ctf_run00504428_orbit0000128764_tf0000001006.root
o2_ctf_run00504428_orbit0000129532_tf0000001012.root
o2_ctf_run00504428_orbit0000159356_tf0000001245.root
o2_ctf_run00504428_orbit0000175484_tf0000001371.root
o2_ctf_run00504428_orbit4294967295_tf0000000004.root
o2_ctf_run00504428_orbit4294967295_tf0000000005.root
o2_ctf_run00504428_orbit4294967295_tf0000000006.root
o2_ctf_run00504428_orbit4294967295_tf000000000

0

### Generating file lists

For systematic searches, it's probably better to retrieve the results in some Python data structures. Maybe they could even be converted to pandas data frames...

In [4]:
import json
from pprint import pprint
res = j.run('ls /alice/data/2021/OCT_TRD/504428/raw')
files = [x['path'] for x in res.ansdict['results']]
display(files)

['/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000000124_tf0000000001.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000001020_tf0000000008.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000002428_tf0000000019.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000002812_tf0000000022.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000005756_tf0000000045.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000006268_tf0000000049.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000007036_tf0000000055.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000127356_tf0000000995.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000127612_tf0000000997.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000127868_tf0000000999.root',
 '/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit00001285

In [5]:
# filename = "/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit4294967295_tf0000001073.root"
j.ProcessMsg(f"whereis {files[0]}")

the file o2_ctf_run00504428_orbit0000000124_tf0000000001.root is in

	 SE => ALICE::CERN::EOSALICEO2  pfn => root://eosaliceo2.cern.ch:1094//raw/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000000124_tf0000000001.root


0

### Copying files

It's straightforward to download a file with `alien.py`

In [7]:
j.ProcessMsg(f"cp {files[0]} file:/tmp/bla.root")

jobID: 1/1 >>> Start
jobID: 1/1 >>> STATUS OK >>> SPEED 108.51 MiB/s
Succesful jobs (1st try): 1/1


0

In [55]:
f1 = ROOT.TFile("/tmp/bla.root")
f1.ls()

TFile**		/tmp/bla.root	
 TFile*		/tmp/bla.root	
  KEY: TTree	ctf;1	O2 CTF tree


In [66]:
j.ProcessMsg(f"ls -l /alice/cern.ch/user/t/tdietel/digfilter_digflt1/digfilter_digflt1.root")

-rwxr-xr-x    tdietel  tdietel        21937 Nov 27 08:39    digfilter_digflt1.root


0

ROOT Implementation of AliEn: TJAlien
===============================

TJAlien is the implementation in ROOT, and would be preferable to open a file directly from the grid, rather than copying it locally before opening it - although that assumes that xrootd access does not copy it locally itself.

**THIS PART IS BROKEN, BUT I LEAVE IT IN AS A STARTING POINT**

In [6]:
import ROOT
ali = ROOT.TJAlien("alien:///alice/data")
ali.Ls().Print()


Welcome to JupyROOT 6.24/06


In [10]:
#  alien = ROOT.TGridConnect("alien", ROOT.gSystem.Getenv("USER"), "", "-domain=cern.ch");

In [11]:
filename = "/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit4294967295_tf0000001073.root"
ali.Ls("/alice/data/2021/OCT_TRD/504428/raw/").Print()

( 000000 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000001 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000002 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000003 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000004 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000005 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000006 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000007 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000008 ) LFN: .../(null)                                             Size[Bytes]:     (null)   GUID: (null)
( 000009 )

Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <TUnixSystem::BaseName>: name = 0
Error in <T

In [42]:
f2 = ROOT.TJAlienFile("root://eosaliceo2.cern.ch:1094//raw/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000000124_tf0000000001.root")

Error in <TNetXNGFile::Open>: [ERROR] Server responded with an error: [3010] Unable to open /raw/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit0000000124_tf0000000001.root; Permission denied



In [10]:
f = ROOT.TJAlienFile("/alice/data/2021/OCT_TRD/504428/raw/o2_ctf_run00504428_orbit4294967295_tf0000001073.root")

Error in <TNetXNGFile::Open>: [ERROR] Operation expired


In [ ]:
f.ls()

In [ ]:
h = ROOT.TH1F("bla","foo", 40,0., 10.)

In [ ]:
cnv = ROOT.TCanvas()
h.Draw()
cnv.Draw()

In [10]:
hit = ROOT.o2.trd.Hit(1,2,3,4,42,666,1)
print(hit)

In [ ]:
alien = ROOT.TJAlien("alien:///alice/data")

In [ ]:
alien.Ls()